In [21]:
import pandas as pd
import os
import glob

print("Imports Successful!")

Imports Successful!


In [22]:
# Load demographic and diagnosis data
demographic_columns = ['Patient number', 'Age', 'Sex', 'Adult BMI (kg/m2)', 'Child Weight (kg)', 'Child Height (cm)']
diagnosis_columns = ['Patient number', 'Diagnosis']
demographics = pd.read_csv('/Users/omama/Documents/Portfolio/respiratory_classification/respiratory-sound-database/demographic_info.txt', delim_whitespace=True, names=demographic_columns, header=None)
diagnosis = pd.read_csv('/Users/omama/Documents/Portfolio/respiratory_classification/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/patient_diagnosis.csv', sep=',', names=diagnosis_columns, header=None)

demographics.head()


/var/folders/fs/ck8sj3v57fnfx4pymvf1b9jw0000gn/T/ipykernel_83711/545876455.py:4: FutureWarning: The 'delim_whitespace' keyword in pd.read_csv is deprecated and will be removed in a future version. Use ``sep='\s+'`` instead
  demographics = pd.read_csv('/Users/omama/Documents/Portfolio/respiratory_classification/respiratory-sound-database/demographic_info.txt', delim_whitespace=True, names=demographic_columns, header=None)


,Patient number,Age,Sex,Adult BMI (kg/m2),Child Weight (kg),Child Height (cm)
0,101,3.00,F,NaN,19.0,99.0
1,102,0.75,F,NaN,9.8,73.0
2,103,70.00,F,33.00,NaN,NaN
3,104,70.00,F,28.47,NaN,NaN
4,105,7.00,F,NaN,32.0,135.0


In [23]:
diagnosis.head()

,Patient number,Diagnosis
0,101,URTI
1,102,Healthy
2,103,Asthma
3,104,COPD
4,105,URTI


In [35]:
demographics['Patient number'].dtype
diagnosis['Patient number'].dtype

dtype('int64')

In [24]:
# Directory containing audio and annotation files
data_dir = '/Users/omama/Documents/Portfolio/respiratory_classification/respiratory-sound-database/Respiratory_Sound_Database/Respiratory_Sound_Database/audio_and_txt_files'

# List all .wav files
audio_files = glob.glob(os.path.join(data_dir, '*.wav'))


In [25]:
# Print the number of audio files found
print(f"Number of audio files found: {len(audio_files)}")

Number of audio files found: 920


In [26]:
# Initialize lists to store extracted data
file_info = []

# Loop through each audio file and extract information
for audio_file in audio_files:
    filename = os.path.basename(audio_file)
    parts = filename.split('_')
    
    # Debugging: Print the parts of the filename
    print(f"Filename: {filename}, Parts: {parts}")
    
    if len(parts) == 5:
        patient_number = parts[0]
        recording_index = parts[1]
        chest_location = parts[2]
        acquisition_mode = parts[3]
        recording_equipment = parts[4].rsplit('.', 1)[0]  # Remove the .wav extension
        
        # Add file info to list
        file_info.append([filename, patient_number, recording_index, chest_location, acquisition_mode, recording_equipment])
    else:
        # Debugging: Print filenames that do not match the expected pattern
        print(f"Skipping filename with unexpected format: {filename}")

# Debugging: Check the content of the file_info list
print(f"Extracted file information: {file_info}")

# Create a dataframe from the extracted information
file_info_df = pd.DataFrame(file_info, columns=['filename', 'patient_number', 'recording_index', 'chest_location', 'acquisition_mode', 'recording_equipment'])


Filename: 122_2b1_Tc_mc_LittC2SE.wav, Parts: ['122', '2b1', 'Tc', 'mc', 'LittC2SE.wav']
Filename: 113_1b1_Lr_sc_Litt3200.wav, Parts: ['113', '1b1', 'Lr', 'sc', 'Litt3200.wav']
Filename: 178_1b3_Pl_mc_AKGC417L.wav, Parts: ['178', '1b3', 'Pl', 'mc', 'AKGC417L.wav']
Filename: 220_1b1_Tc_mc_LittC2SE.wav, Parts: ['220', '1b1', 'Tc', 'mc', 'LittC2SE.wav']
Filename: 176_1b3_Tc_mc_AKGC417L.wav, Parts: ['176', '1b3', 'Tc', 'mc', 'AKGC417L.wav']
Filename: 154_2b4_Ll_mc_AKGC417L.wav, Parts: ['154', '2b4', 'Ll', 'mc', 'AKGC417L.wav']
Filename: 117_1b3_Tc_mc_LittC2SE.wav, Parts: ['117', '1b3', 'Tc', 'mc', 'LittC2SE.wav']
Filename: 218_1b1_Lr_sc_Meditron.wav, Parts: ['218', '1b1', 'Lr', 'sc', 'Meditron.wav']
Filename: 185_1b1_Ll_sc_Litt3200.wav, Parts: ['185', '1b1', 'Ll', 'sc', 'Litt3200.wav']
Filename: 130_2b4_Pl_mc_AKGC417L.wav, Parts: ['130', '2b4', 'Pl', 'mc', 'AKGC417L.wav']
Filename: 156_8b3_Pl_mc_AKGC417L.wav, Parts: ['156', '8b3', 'Pl', 'mc', 'AKGC417L.wav']
Filename: 172_1b5_Al_mc_AKGC417L

In [27]:
file_info_df.head()

,filename,patient_number,recording_index,chest_location,acquisition_mode,recording_equipment
0,122_2b1_Tc_mc_LittC2SE.wav,122,2b1,Tc,mc,LittC2SE
1,113_1b1_Lr_sc_Litt3200.wav,113,1b1,Lr,sc,Litt3200
2,178_1b3_Pl_mc_AKGC417L.wav,178,1b3,Pl,mc,AKGC417L
3,220_1b1_Tc_mc_LittC2SE.wav,220,1b1,Tc,mc,LittC2SE
4,176_1b3_Tc_mc_AKGC417L.wav,176,1b3,Tc,mc,AKGC417L


In [36]:
# Convert 'patient_number' to string
file_info_df['patient_number'] = file_info_df['patient_number'].astype(int)
file_info_df['patient_number'].dtype


dtype('int64')

In [28]:
# Initialize lists to store annotation data
annotations = []

# Iterate over annotation files
for audio_file in audio_files:
    annotation_file = audio_file.replace('.wav', '.txt')
    with open(annotation_file, 'r') as f:
        for line in f:
            start, end, crackles, wheezes = line.strip().split('\t')
            annotations.append([os.path.basename(audio_file), float(start), float(end), int(crackles), int(wheezes)])

# Create a dataframe for annotations
annotations_df = pd.DataFrame(annotations, columns=['filename', 'start', 'end', 'crackles', 'wheezes'])

In [29]:
annotations_df[0:20]

,filename,start,end,crackles,wheezes
0,122_2b1_Tc_mc_LittC2SE.wav,0.0070,3.2640,0,1
1,122_2b1_Tc_mc_LittC2SE.wav,3.2640,7.3360,0,1
2,122_2b1_Tc_mc_LittC2SE.wav,7.3360,10.6360,0,1
3,122_2b1_Tc_mc_LittC2SE.wav,10.6360,14.3640,0,1
4,122_2b1_Tc_mc_LittC2SE.wav,14.3640,18.1360,0,1
5,122_2b1_Tc_mc_LittC2SE.wav,18.1360,19.9790,0,0
6,113_1b1_Lr_sc_Litt3200.wav,0.0000,1.8667,0,0
7,113_1b1_Lr_sc_Litt3200.wav,1.8667,3.3948,0,0
8,113_1b1_Lr_sc_Litt3200.wav,3.3948,6.0796,0,0
9,113_1b1_Lr_sc_Litt3200.wav,6.0796,7.5795,0,0


In [39]:
# Merge the dataframes on the 'filename' column
merged_df = pd.merge(file_info_df, annotations_df, on='filename')

# Merge with demographic data
merged_df = merged_df.merge(demographics, left_on='patient_number', right_on='Patient number', how='left')

# Merge with diagnosis data
merged_df = merged_df.merge(diagnosis, left_on='patient_number', right_on='Patient number', how='left')

# Save to a CSV file for future use
merged_df.to_csv('/Users/omama/Documents/Portfolio/respiratory_classification/data/processed_respiratory_data.csv', index=False)

# Display the first few rows of the final dataframe
print(merged_df.head())

                     filename  patient_number recording_index chest_location  \
0  122_2b1_Tc_mc_LittC2SE.wav             122             2b1             Tc   
1  122_2b1_Tc_mc_LittC2SE.wav             122             2b1             Tc   
2  122_2b1_Tc_mc_LittC2SE.wav             122             2b1             Tc   
3  122_2b1_Tc_mc_LittC2SE.wav             122             2b1             Tc   
4  122_2b1_Tc_mc_LittC2SE.wav             122             2b1             Tc   

  acquisition_mode recording_equipment   start     end  crackles  wheezes  \
0               mc            LittC2SE   0.007   3.264         0        1   
1               mc            LittC2SE   3.264   7.336         0        1   
2               mc            LittC2SE   7.336  10.636         0        1   
3               mc            LittC2SE  10.636  14.364         0        1   
4               mc            LittC2SE  14.364  18.136         0        1   

   Patient number_x   Age Sex  Adult BMI (kg/m2)  Child 